In [ ]:
import pdfplumber
import re
from collections import Counter

def analyze_font_sizes(pdf):
    """
    Scans the PDF to determine the most common font size (Body Text)
    and identify sizes likely to be headers.
    """
    font_sizes = []
    for page in pdf.pages:
        # Get all words and their heights
        words = page.extract_words(extra_attrs=["size"])
        for w in words:
            if w['size']:
                # Round to nearest integer to group similar sizes
                font_sizes.append(round(w['size']))
    
    if not font_sizes:
        return 10, {} # Fallback

    # The most common font size is likely the Body Text
    size_counts = Counter(font_sizes)
    body_font_size = size_counts.most_common(1)[0][0]
    
    return body_font_size

def is_footer_or_header(text, page_height, y_position):
    """
    Simple heuristic to filter out page numbers and copyright footers 
    based on vertical position (top 5% or bottom 10% of page).
    """
    if y_position < (page_height * 0.05) or y_position > (page_height * 0.90):
        # Specific filters for the document you provided
        skip_keywords = ["Page", "Copyright", "UnitedHealthcare", "Proprietary Information", "Effective"]
        if any(keyword in text for keyword in skip_keywords):
            return True
    return False

def list_to_markdown_table(table_data):
    """
    Converts a list of lists (from pdfplumber) into a Markdown table.
    """
    if not table_data:
        return ""
    
    # Clean None values
    clean_data = [[str(cell).replace('\n', ' ').strip() if cell else "" for cell in row] for row in table_data]
    
    # Create header row
    header = "| " + " | ".join(clean_data[0]) + " |"
    separator = "| " + " | ".join(["---"] * len(clean_data[0])) + " |"
    
    body = ""
    for row in clean_data[1:]:
        body += "\n| " + " | ".join(row) + " |"
        
    return f"\n{header}\n{separator}{body}\n"

def convert_pdf_to_md(pdf_path, md_path):
    print(f"Processing {pdf_path}...")
    
    with pdfplumber.open(pdf_path) as pdf:
        body_font_size = analyze_font_sizes(pdf)
        print(f"Detected Body Font Size: ~{body_font_size}pt")
        
        markdown_content = []
        
        for i, page in enumerate(pdf.pages):
            print(f"Scanning Page {i+1}...")
            
            # 1. Extract Tables first to avoid duplicating text
            tables = page.find_tables()
            table_bboxes = [t.bbox for t in tables] # Get bounding boxes to exclude this text later
            
            extracted_tables = []
            for table in page.extract_tables():
                extracted_tables.append(list_to_markdown_table(table))
            
            # 2. Extract Text Line by Line
            text_content = []
            
            # Extract words with layout info
            words = page.extract_words(extra_attrs=["size", "top", "bottom"])
            
            # Group words into lines based on "top" position (with small tolerance)
            lines = {}
            for w in words:
                # Check if this word is inside a table; if so, skip it
                in_table = False
                w_center_x = (w['x0'] + w['x1']) / 2
                w_center_y = (w['top'] + w['bottom']) / 2
                
                for bbox in table_bboxes:
                    if (bbox[0] < w_center_x < bbox[2]) and (bbox[1] < w_center_y < bbox[3]):
                        in_table = True
                        break
                if in_table:
                    continue

                # Group by Y position (rounded to combine words on same line)
                y_pos = round(w['top'])
                if y_pos not in lines:
                    lines[y_pos] = []
                lines[y_pos].append(w)
            
            # Sort lines by vertical position
            sorted_y = sorted(lines.keys())
            
            for y in sorted_y:
                line_words = lines[y]
                # Reconstruct the line text
                line_text = " ".join([w['text'] for w in line_words])
                
                # Check if it's a footer/header garbage
                if is_footer_or_header(line_text, page.height, y):
                    continue
                
                # Determine Heading Level based on max font size in the line
                max_size = max([w['size'] for w in line_words])
                
                # Heuristics for formatting
                if max_size > body_font_size * 1.5:
                    prefix = "# " # H1
                elif max_size > body_font_size * 1.1:
                    prefix = "## " # H2
                else:
                    prefix = ""
                
                # Bold logic (simplified: if the whole line is bold font)
                # pdfplumber font names usually contain "Bold"
                is_bold = all("Bold" in w.get("fontname", "") for w in line_words)
                if is_bold and prefix == "":
                    line_text = f"**{line_text}**"
                
                text_content.append(f"{prefix}{line_text}")

            # 3. Assemble Page Content
            # We append text, then insert tables at the end of the page (simplification)
            # Or interleave them if we tracked Y-positions strictly. 
            # For this script, we append tables after text for cleaner reading if strict flow isn't critical.
            
            page_md = "\n".join(text_content)
            for tbl in extracted_tables:
                # Heuristic: append tables at the end of the section
                page_md += f"\n\n{tbl}\n"
                
            markdown_content.append(page_md)

    # Save to file
    with open(md_path, "w", encoding="utf-8") as f:
        full_text = "\n\n---\n\n".join(markdown_content)
        f.write(full_text)
    
    print(f"Successfully saved to {md_path}")

# --- Execution ---
if __name__ == "__main__":
    # Replace with your actual file name
    input_pdf = "spinal_pain.pdf" 
    output_md = "spinal_pain.md"
    
    try:
        convert_pdf_to_md(input_pdf, output_md)
    except FileNotFoundError:
        print(f"Error: Could not find {input_pdf}. Make sure the file is in the same folder.")